In [1]:
!pip install numpy pandas scikit-learn xgboost deap sentence-transformers
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error, r2_score
from deap import base, creator, tools, algorithms
import random
import warnings
warnings.filterwarnings("ignore")

# Load dataset
dataset_path = "/content/extended_5000_dataset.csv"  # Replace with your dataset path
data = pd.read_csv(dataset_path)

# Encode categorical features
label_encoders = {}
for col in data.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])
    label_encoders[col] = le

# Define target and features
y = data['TOTAL IPC CRIMES']
X = data.drop(columns=['TOTAL IPC CRIMES'])

# Feature correlation and initial cleaning
correlations = X.corr().abs()
upper_tri = correlations.where(np.triu(np.ones(correlations.shape), k=1).astype(bool))
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.75)]
X = X.drop(columns=to_drop)

# Train-test split
y_binned = pd.qcut(y, q=4, labels=False, duplicates='drop')
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    stratify=y_binned, random_state=42)

# Standardize features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X.columns)

# GA for Feature Selection
def evaluate_feature_selection(individual):
    selected_features = [col for i, col in enumerate(X.columns) if individual[i] == 1]
    if not selected_features:
        return -np.inf,
    model = Ridge(alpha=50.0)
    scores = cross_val_score(model, X_train_scaled[selected_features], y_train,
                           cv=10, scoring='r2')
    feature_penalty = -0.005 * sum(individual)
    return np.mean(scores) + feature_penalty,

creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)
toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual,
                toolbox.attr_bool, n=len(X.columns))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)
toolbox.register("evaluate", evaluate_feature_selection)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=4)

population = toolbox.population(n=60)
hof = tools.HallOfFame(1)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("max", np.max)

population, log = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.3,
                                    ngen=50, stats=stats, halloffame=hof, verbose=True)

best_features = [col for i, col in enumerate(X.columns) if hof[0][i] == 1]
X_train_selected = X_train_scaled[best_features]
X_test_selected = X_test_scaled[best_features]

print("Selected Features (GA):", best_features)

# Train models with adjusted hyperparameters
models = {
    "Ridge": Ridge(alpha=50.0),
    "Lasso": Lasso(alpha=25.0),
    "RandomForest": RandomForestRegressor(
        n_estimators=200,
        max_depth=8,
        min_samples_split=100,
        min_samples_leaf=50,
        random_state=42
    ),
    "GradientBoosting": GradientBoostingRegressor(
        n_estimators=200,
        learning_rate=0.01,
        max_depth=5,
        random_state=42
    ),
    "XGBoost": XGBRegressor(
        n_estimators=200,
        learning_rate=0.01,
        max_depth=5,
        subsample=0.7,
        colsample_bytree=0.5,
        random_state=42
    )
}

# Train all models
trained_models = {}
model_results = {}
for name, model in models.items():
    model.fit(X_train_selected, y_train)
    y_pred_train = model.predict(X_train_selected)
    y_pred_test = model.predict(X_test_selected)
    trained_models[name] = model
    model_results[name] = {
        "Train MAE": mean_absolute_error(y_train, y_pred_train),
        "Test MAE": mean_absolute_error(y_test, y_pred_test),
        "R^2": r2_score(y_test, y_pred_test)
    }

print("Model Accuracy Results:")
print(pd.DataFrame(model_results).T)

# Zero-Shot Learning Prediction
def predict_unseen_crime(unseen_crime):
    from sentence_transformers import SentenceTransformer
    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    known_crimes = best_features  # Ensuring only GA-selected features are used
    known_embeddings = model.encode(known_crimes)
    unseen_embedding = model.encode([unseen_crime])[0]

    similarities = np.dot(known_embeddings, unseen_embedding) / (np.linalg.norm(known_embeddings, axis=1) * np.linalg.norm(unseen_embedding))
    top_matches = np.argsort(similarities)[-3:][::-1]
    valid_features = [known_crimes[i] for i in top_matches]

    if valid_features:
        # Create a DataFrame for the prediction with all selected features
        X_pred = pd.DataFrame(np.zeros((1, len(best_features))), columns=best_features)

        # Fill missing features with their mean value from the training set
        for feature in valid_features:
            if feature in X_train_selected.columns:
                X_pred[feature] = X_train_selected[feature].mean()

        # Ensure all columns from best_features are present in the DataFrame
        for feature in best_features:
            if feature not in valid_features:
                X_pred[feature] = X_train_selected[feature].mean()  # Fill missing features with the mean

        weighted_crime_rate = trained_models["RandomForest"].predict(X_pred)[0]
        return weighted_crime_rate
    return None

# Example ZSL prediction
unseen_crime = "Sedition"
predicted_rate = predict_unseen_crime(unseen_crime)
if predicted_rate is not None:
    print(f"Predicted crime rate for {unseen_crime}: {predicted_rate:.2f}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.6/135.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 76.0 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Predicted crime rate for Sedition: 118825.69


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV, cross_val_score
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline
import seaborn as sns
import matplotlib.pyplot as plt

# Load dataset
df = pd.read_csv("/content/extended_5000_dataset.csv")
df.columns = df.columns.str.strip()

# Target column
target_column = "STATE/UT"
if target_column not in df.columns:
    raise KeyError(f"Column '{target_column}' not found. Check column names.")

# Encode target column
if df[target_column].dtype == 'object':
    le = LabelEncoder()
    df[target_column] = le.fit_transform(df[target_column])

# Feature & Target split
X = df.drop(columns=[target_column])
y = df[target_column]

# Select only numeric features
X = X.select_dtypes(include=[np.number])

# Train-test split (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# Define models and hyperparameters
model_params = {
    'Logistic Regression': {
        'model': LogisticRegression(max_iter=2000, class_weight='balanced', solver='saga', random_state=42),
        'params': {'C': [0.01, 0.1, 1, 10]}
    },
    'Random Forest': {
        'model': RandomForestClassifier(random_state=42, class_weight='balanced'),
        'params': {'n_estimators': [100, 200], 'max_depth': [None, 10, 20]}
    },
    'Gradient Boosting': {
        'model': GradientBoostingClassifier(random_state=42),
        'params': {'n_estimators': [100, 200], 'learning_rate': [0.05, 0.1], 'max_depth': [3, 5]}
    },
    'Support Vector Machine': {
        'model': SVC(probability=True, class_weight='balanced', random_state=42),
        'params': {'C': [0.1, 1, 10], 'kernel': ['rbf', 'poly'], 'gamma': ['scale', 'auto']}
    },
    'K-Nearest Neighbors': {
        'model': KNeighborsClassifier(),
        'params': {'n_neighbors': [5, 10, 15], 'weights': ['uniform', 'distance']}
    },
    'Decision Tree': {
        'model': DecisionTreeClassifier(random_state=42, class_weight='balanced'),
        'params': {'max_depth': [None, 10, 20], 'min_samples_leaf': [1, 5, 10]}
    }
}

# Scaling, feature selection & SMOTE inside pipeline
results = []

for model_name, mp in model_params.items():
    pipeline = ImbPipeline([
        ('scaler', StandardScaler()),
        ('selector', SelectKBest(f_classif, k=10)),
        ('smote', SMOTE(random_state=42)),
        ('model', mp['model'])
    ])

    clf = GridSearchCV(pipeline, param_grid={'model__' + k: v for k, v in mp['params'].items()},
                       cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
                       scoring='accuracy', n_jobs=-1)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    results.append({'Model': model_name, 'Best_Params': clf.best_params_, 'Accuracy': acc})

    print(f"\n====== {model_name} ======")
    print("Best Parameters:", clf.best_params_)
    print("Test Accuracy:", acc)
    print("Classification Report:\n", classification_report(y_test, y_pred))
    cm = confusion_matrix(y_test, y_pred)
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f"{model_name} Confusion Matrix")
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()

# Display final comparison table
results_df = pd.DataFrame(results).sort_values(by='Accuracy', ascending=False)
print("\n\n### Final Comparison Table ###")
print(results_df.to_string(index=False))